# Task 1 - Modeling with knodle

In [1]:
import os
import numpy as np
import knodle
import torch
import joblib
from transformers import AutoModelForSequenceClassification, AdamW
from knodle.trainer import AutoTrainer, AutoConfig, baseline, MajorityVoteTrainer, multi_trainer
from knodle.trainer.baseline.config import MajorityConfig

2023-01-23 15:32:29,937 root         INFO     Initalized logger
/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-23 15:32:31.059123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 15:32:32.393630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-23 15:32:32.393906: W tensorflow/compiler/xla/stream_executor/platform/defaul

In [3]:
x_train = joblib.load('./data/x_train.lib')
t_train = joblib.load('./data/t_matrix.lib')
z_train = joblib.load('./data/z_matrix.lib')
x_test = joblib.load('./data/x_test.lib')
y_test = joblib.load('./data/y_test.lib')
x_dev = joblib.load('./data/x_dev.lib')
y_dev = joblib.load('./data/y_dev.lib')

Here are collected some preliminary modelling attempts, that are not fully adequate for the challenge at hand.

In [4]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

trainer_type = "majority"
custom_model_config = MajorityConfig(
    optimizer=AdamW,
    lr=1e-4,
    batch_size=16,
    epochs=2
    #,multi_label = True
    ,other_class_id = 0
    ,filter_non_labelled=False

)


print(custom_model_config.__dict__)


trainer_2 = MajorityVoteTrainer(
    model=model,
    mapping_rules_labels_t=np.array(t_train.to_dense()),
    model_input_x=x_train,
    rule_matches_z=np.array(z_train.to_dense()),
    #dev_model_input_x=x_dev,
    #dev_gold_labels_y=y_dev,
    trainer_config=custom_model_config,
)

trainer_2.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

{'seed': None, 'caching_suffix': '', 'caching_folder': '/Usage/Florian/Studium/DigitalHumanities/Semester4/DataAnalysisProject/git/Weak-Supervision-in-the-Medical-Domain/Task 1/cache', 'saved_models_dir': '/Usage/Florian/Studium/DigitalHumanities/Semester4/DataAnalysisProject/git/Weak-Supervision-in-the-Medical-Domain/Task 1/cache', 'criterion': <function cross_entropy_with_probs at 0x7f194ab2f5f0>, 'lr': 0.0001, 'batch_size': 16, 'output_classes': 2, 'grad_clipping': None, 'device': device(type='cuda'), 'epochs': 2, 'optimizer': <class 'transformers.optimization.AdamW'>, 'class_weights': tensor([1., 1.]), 'filter_non_labelled': True, 'other_class_id': None, 'evaluate_with_other_class': False, 'ids2labels': None, 'max_rules': None, 'min_coverage': None, 'drop_rules': False, 'use_probabilistic_labels': True, 'probability_threshold': None}


2023-01-16 19:56:15,975 knodle.transformation.rule_reduction INFO     No filtering criteria ('max_rule' or 'min_coverage' for rule specified, returning the original rule matches.
/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/knodle/transformation/majority.py:78: RuntimeWarning: invalid value encountered in true_divide
  rule_counts_probs = rule_counts / rule_counts.sum(axis=1).reshape(-1, 1)
2023-01-16 19:56:26,052 knodle.trainer.trainer INFO     ======================================
2023-01-16 19:56:26,053 knodle.trainer.trainer INFO     Training starts
2023-01-16 19:56:26,054 knodle.trainer.trainer INFO     =============================

In [5]:
eval_dict, _ = trainer.test(x_test, y_test)
print(f"Accuracy: {eval_dict.get('accuracy')}")

100%|██████████| 794/794 [00:08<00:00, 93.54it/s]

Accuracy: 0.9018820379557445



/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resul

In [17]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

trainer_type = "majority"
custom_model_config = AutoConfig.create_config(
    name=trainer_type,
    optimizer=AdamW,
    lr=1e-4,
    batch_size=16,
    epochs=2
    #multi_label = True,
    #other_class_id = 0,
    #,filter_non_labeled = False, 

)

print(custom_model_config.__dict__)

trainer_2 = AutoTrainer(
    name="majority",
    model=model,
    mapping_rules_labels_t=np.array(t_train.to_dense()),
    model_input_x=x_train,
    rule_matches_z=np.array(z_train.to_dense()),
    dev_model_input_x=x_dev,
    dev_gold_labels_y=y_dev,
    trainer_config=custom_model_config,
)

trainer_2.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

{'seed': None, 'caching_suffix': '', 'caching_folder': '/Usage/Florian/Studium/DigitalHumanities/Semester4/DataAnalysisProject/git/Weak-Supervision-in-the-Medical-Domain/Task 1/cache', 'saved_models_dir': '/Usage/Florian/Studium/DigitalHumanities/Semester4/DataAnalysisProject/git/Weak-Supervision-in-the-Medical-Domain/Task 1/cache', 'criterion': <function cross_entropy_with_probs at 0x7f194ab2f5f0>, 'lr': 0.0001, 'batch_size': 16, 'output_classes': 2, 'grad_clipping': None, 'device': device(type='cuda'), 'epochs': 2, 'optimizer': <class 'transformers.optimization.AdamW'>, 'class_weights': tensor([1., 1.]), 'filter_non_labelled': True, 'other_class_id': None, 'evaluate_with_other_class': False, 'ids2labels': None, 'max_rules': None, 'min_coverage': None, 'drop_rules': False, 'use_probabilistic_labels': True, 'probability_threshold': None}


2023-01-16 22:35:09,913 knodle.transformation.rule_reduction INFO     No filtering criteria ('max_rule' or 'min_coverage' for rule specified, returning the original rule matches.
2023-01-16 22:39:09,146 knodle.trainer.trainer INFO     ======================================
2023-01-16 22:39:09,147 knodle.trainer.trainer INFO     Training starts
2023-01-16 22:39:09,147 knodle.trainer.trainer INFO     ======================================


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.